# Prueba Tecnica DataKnow

In [7]:
import pandas as pd
from io import StringIO

## 1. Carga de información 

Se lee el archivo txt

In [5]:
with open('../Datos3/OFEI1204.txt', 'r') as file:
    txt_data = file.read()

print(txt_data)

Ofertas Iniciales para: 2017-12-04


AGENTE: AES CHIVOR
CHIVOR1 , D,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125
CHIVOR2 , D,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125
CHIVOR3 , D,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125
CHIVOR4 , D,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125
CHIVOR5 , D,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125
CHIVOR6 , D,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125,  125
CHIVOR7 

Se divide el contenido por agente

In [28]:
agentes = []
dfs = []

secciones = txt_data.split("AGENTE:")
for seccion in secciones[1:]:
    agente, tabla_str = seccion.split("\n", 1)
    agentes.append(agente.strip())
    tabla = pd.read_csv(StringIO(tabla_str), delimiter=',', skipinitialspace=True, header=None)
    tabla['AGENTE'] = agente.strip()
    dfs.append(tabla)

df_final = pd.concat(dfs, ignore_index=True)
df_final[df_final["AGENTE"]=="EEPPM"]

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,AGENTE
67,ESMERALDA1,D,14,14.00,14.00,14.00,14.00,14.00,14.00,14.00,...,14.00,14.00,14.00,14.00,14.00,14.00,14.00,14.00,14.00,EEPPM
68,ESMERALDA2,D,14,14.00,14.00,14.00,14.00,14.00,14.00,14.00,...,14.00,14.00,14.00,14.00,14.00,14.00,14.00,14.00,14.00,EEPPM
69,GUATAPE1,D,70,70.00,70.00,70.00,70.00,70.00,70.00,70.00,...,70.00,70.00,70.00,70.00,70.00,70.00,70.00,70.00,70.00,EEPPM
70,GUATAPE2,D,70,70.00,70.00,70.00,70.00,70.00,70.00,70.00,...,70.00,70.00,70.00,70.00,70.00,70.00,70.00,70.00,70.00,EEPPM
71,GUATAPE3,D,70,70.00,70.00,70.00,70.00,70.00,70.00,70.00,...,70.00,70.00,70.00,70.00,70.00,70.00,70.00,70.00,70.00,EEPPM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,RIOGRANDE,D,0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.30,...,0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.30,0.30,EEPPM
161,MUNICIPAL,D,1.23,1.23,1.23,1.23,1.23,1.23,1.23,1.23,...,1.23,1.23,1.23,1.23,1.23,1.23,1.23,1.23,1.23,EEPPM
162,PORCEIIIMENOR,D,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,EEPPM
163,COGENERADORCOLTEJER1,D,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,EEPPM


Se procesan solo los registros Tipo D

In [36]:
df_filtrado_tipo_D = df_final[df_final[1]=="D"].reset_index(drop=True)
df_filtrado_tipo_D

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,AGENTE
0,CHIVOR1,D,125.0,125.00,125.00,125.00,125.00,125.00,125.00,125.00,...,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,AES CHIVOR
1,CHIVOR2,D,125.0,125.00,125.00,125.00,125.00,125.00,125.00,125.00,...,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,AES CHIVOR
2,CHIVOR3,D,125.0,125.00,125.00,125.00,125.00,125.00,125.00,125.00,...,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,AES CHIVOR
3,CHIVOR4,D,125.0,125.00,125.00,125.00,125.00,125.00,125.00,125.00,...,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,AES CHIVOR
4,CHIVOR5,D,125.0,125.00,125.00,125.00,125.00,125.00,125.00,125.00,...,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,AES CHIVOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,SANTIAGO,D,1.52,1.52,1.52,1.52,1.52,1.52,1.52,1.52,...,1.52,1.52,1.52,1.52,1.52,1.52,1.52,1.52,1.52,VATIA S.A.
301,INCAUCA1,D,8.5,8.50,8.50,8.50,8.50,8.50,8.50,8.50,...,8.50,8.50,8.50,8.50,8.50,8.50,8.50,8.50,8.50,VATIA S.A.
302,INGENIOPROVIDENCIA2,D,13.8,13.80,13.80,13.80,13.80,13.80,13.80,13.80,...,13.80,13.80,13.80,13.80,13.80,13.80,13.80,13.80,13.80,VATIA S.A.
303,FLORES1,D,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,ZF CELSIA


Se renombran las columnas y se genera la tabla final

In [74]:
df_filtrado_tipo_D.columns = ['PLANTA'] + ['TIPO'] + [f'Hora_{i-1}' if i > 1 else 'PLANTA' for i in range(2, 26)] + ['AGENTE']
df_tabla_punto_1 = df_filtrado_tipo_D.drop(columns=['TIPO'])
columnas_hora = [col for col in df_tabla_punto_1.columns if col.startswith('Hora')]
df_tabla_punto_1[columnas_hora] = df_tabla_punto_1[columnas_hora].astype(float)
df_tabla_punto_1.to_csv("../resultados/tabla_punto_1.csv")

df_tabla_punto_1

,PLANTA,Hora_1,Hora_2,Hora_3,Hora_4,Hora_5,Hora_6,Hora_7,Hora_8,Hora_9,...,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23,Hora_24,AGENTE
0,CHIVOR1,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,...,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,AES CHIVOR
1,CHIVOR2,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,...,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,AES CHIVOR
2,CHIVOR3,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,...,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,AES CHIVOR
3,CHIVOR4,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,...,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,AES CHIVOR
4,CHIVOR5,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,...,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,125.00,AES CHIVOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,SANTIAGO,1.52,1.52,1.52,1.52,1.52,1.52,1.52,1.52,1.52,...,1.52,1.52,1.52,1.52,1.52,1.52,1.52,1.52,1.52,VATIA S.A.
301,INCAUCA1,8.50,8.50,8.50,8.50,8.50,8.50,8.50,8.50,8.50,...,8.50,8.50,8.50,8.50,8.50,8.50,8.50,8.50,8.50,VATIA S.A.
302,INGENIOPROVIDENCIA2,13.80,13.80,13.80,13.80,13.80,13.80,13.80,13.80,13.80,...,13.80,13.80,13.80,13.80,13.80,13.80,13.80,13.80,13.80,VATIA S.A.
303,FLORES1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,ZF CELSIA


## 2. Manipulación de datos

In [76]:
columnas_seleccionadas = ['Nombre visible Agente', 'AGENTE (OFEI)' , 'CENTRAL (dDEC, dSEGDES, dPRU…)' , 'Tipo de central (Hidro, Termo, Filo, Menor)'] 

df_master_data = pd.read_excel("../Datos3/Datos Maestros VF.xlsx", sheet_name='Master Data Oficial', usecols=columnas_seleccionadas)

df_master_data

,Nombre visible Agente,AGENTE (OFEI),"CENTRAL (dDEC, dSEGDES, dPRU…)","Tipo de central (Hidro, Termo, Filo, Menor)"
0,RIO PIEDRAS,ENERGIA RIO PIEDRAS,MAGUAFRE,M
1,ALEJANDRÍA,PCH ALEJANDRÍA,M_ALEJANDRIA,M
2,EPSA,EPSA(PACIFICO),ALBAN,H
3,EPSA,EPSA(PACIFICO),ALBAN,H
4,EPSA,EPSA(PACIFICO),ALBAN,H
...,...,...,...,...
298,VATIA,VATIA S.A.,MVENTANA_B,M
299,EMGESA,EMGESA S.A.,ZIPAEMG2,T
300,EMGESA,EMGESA S.A.,ZIPAEMG3,T
301,EMGESA,EMGESA S.A.,ZIPAEMG4,T


In [77]:
df_master_data_emgesa = df_master_data[
    (df_master_data["Nombre visible Agente"] == "EMGESA") &
    ((df_master_data["Tipo de central (Hidro, Termo, Filo, Menor)"] == "H") |
     (df_master_data["Tipo de central (Hidro, Termo, Filo, Menor)"] == "T"))
].reset_index(drop=True)

df_master_data_emgesa

,Nombre visible Agente,AGENTE (OFEI),"CENTRAL (dDEC, dSEGDES, dPRU…)","Tipo de central (Hidro, Termo, Filo, Menor)"
0,EMGESA,EMGESA S.A.,BETANIA,H
1,EMGESA,EMGESA S.A.,BETANIA,H
2,EMGESA,EMGESA S.A.,BETANIA,H
3,EMGESA,EMGESA S.A.,CTGEMG1,T
4,EMGESA,EMGESA S.A.,CTGEMG2,T
5,EMGESA,EMGESA S.A.,CTGEMG3,T
6,EMGESA,EMGESA S.A.,ELQUIMBO,H
7,EMGESA,EMGESA S.A.,ELQUIMBO,H
8,EMGESA,EMGESA S.A.,GUAVIO,H
9,EMGESA,EMGESA S.A.,GUAVIO,H


In [78]:
df_dec_1204 = pd.read_csv("../Datos3/dDEC1204.TXT", delimiter=',', encoding='latin1',header=None)
df_dec_1204 = df_dec_1204.drop(df_dec_1204.index[-1])
df_dec_1204

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,AG_ARGOSCTG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AG_ARGOSYUM,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AG_ELCAIRO,0,0,0,1,3,3,3,3,3,...,3,0,0,0,0,1,3,3,3,3
3,AG_REFICAR,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
4,AG_UNIBOL,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,VENEZUE1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
212,ZIPAEMG2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
213,ZIPAEMG3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
214,ZIPAEMG4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
df_dec_1204.columns = ['CENTRAL (dDEC, dSEGDES, dPRU…)'] + [f'Hora_{i}' for i in range(1, 25)]

df_dec_1204

,"CENTRAL (dDEC, dSEGDES, dPRU…)",Hora_1,Hora_2,Hora_3,Hora_4,Hora_5,Hora_6,Hora_7,Hora_8,Hora_9,...,Hora_15,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23,Hora_24
0,AG_ARGOSCTG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AG_ARGOSYUM,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AG_ELCAIRO,0,0,0,1,3,3,3,3,3,...,3,0,0,0,0,1,3,3,3,3
3,AG_REFICAR,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
4,AG_UNIBOL,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,VENEZUE1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
212,ZIPAEMG2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
213,ZIPAEMG3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
214,ZIPAEMG4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
df_merged_emgesa = pd.merge(df_master_data_emgesa, df_dec_1204, on='CENTRAL (dDEC, dSEGDES, dPRU…)', how='left')

df_merged_emgesa

,Nombre visible Agente,AGENTE (OFEI),"CENTRAL (dDEC, dSEGDES, dPRU…)","Tipo de central (Hidro, Termo, Filo, Menor)",Hora_1,Hora_2,Hora_3,Hora_4,Hora_5,Hora_6,...,Hora_15,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23,Hora_24
0,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,364
1,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,364
2,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,364
3,EMGESA,EMGESA S.A.,CTGEMG1,T,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,EMGESA,EMGESA S.A.,CTGEMG2,T,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,EMGESA,EMGESA S.A.,CTGEMG3,T,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,EMGESA,EMGESA S.A.,ELQUIMBO,H,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,85
7,EMGESA,EMGESA S.A.,ELQUIMBO,H,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,85
8,EMGESA,EMGESA S.A.,GUAVIO,H,0,0,0,0,0,0,...,180,180,120,120,997,1002,780,140,0,0
9,EMGESA,EMGESA S.A.,GUAVIO,H,0,0,0,0,0,0,...,180,180,120,120,997,1002,780,140,0,0


In [81]:
columnas_hora = [col for col in df_merged_emgesa.columns if col.startswith('Hora')]
df_merged_emgesa['Total_Horas'] = df_merged_emgesa[columnas_hora].sum(axis=1)
df_tabla_punto_2 = df_merged_emgesa[df_merged_emgesa['Total_Horas']>0]
df_tabla_punto_2.to_csv("../resultados/tabla_punto_2.csv")

df_tabla_punto_2


,Nombre visible Agente,AGENTE (OFEI),"CENTRAL (dDEC, dSEGDES, dPRU…)","Tipo de central (Hidro, Termo, Filo, Menor)",Hora_1,Hora_2,Hora_3,Hora_4,Hora_5,Hora_6,...,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23,Hora_24,Total_Horas
0,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,8736
1,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,8736
2,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,8736
6,EMGESA,EMGESA S.A.,ELQUIMBO,H,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,2040
7,EMGESA,EMGESA S.A.,ELQUIMBO,H,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,2040
8,EMGESA,EMGESA S.A.,GUAVIO,H,0,0,0,0,0,0,...,180,120,120,997,1002,780,140,0,0,4325
9,EMGESA,EMGESA S.A.,GUAVIO,H,0,0,0,0,0,0,...,180,120,120,997,1002,780,140,0,0,4325
10,EMGESA,EMGESA S.A.,GUAVIO,H,0,0,0,0,0,0,...,180,120,120,997,1002,780,140,0,0,4325
11,EMGESA,EMGESA S.A.,GUAVIO,H,0,0,0,0,0,0,...,180,120,120,997,1002,780,140,0,0,4325
12,EMGESA,EMGESA S.A.,GUAVIO,H,0,0,0,0,0,0,...,180,120,120,997,1002,780,140,0,0,4325


## Punto 3